<a href="https://colab.research.google.com/github/kazuma2002/OpenScienceDataChallenge/blob/main/Sentinel1_DataAquisition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization
import ipyleaflet
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns

# Data Science
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Feature Engineering
from sklearn.model_selection import train_test_split, KFold

# Machine Learning
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import r2_score


# Planetary Computer Tools
import pystac
import pystac_client
import odc
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
from odc.stac import stac_load
import planetary_computer as pc

#Please pass your API key here
pc.settings.set_subscription_key('c3ed0e9c76f44014a77ef43b454f6747')

# Others
import requests
import rich.table
from itertools import cycle
from tqdm import tqdm
tqdm.pandas()
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

#Additionals
!pip install mlxtend
from sklearn.model_selection import GridSearchCV
import multiprocessing
from sklearn.model_selection import cross_val_score
from mlxtend.regressor import StackingCVRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xarray as xr
from skimage import exposure
from skimage.feature import graycomatrix, graycoprops

In [ ]:
crop_yield_data = pd.read_csv("Crop_Yield_Data_challenge_2.csv")
crop_yield_data.head()

Sentinel-1

In [ ]:
def get_sentinel_data(longitude, latitude, season,assests):

    bands_of_interest = assests
    if season == 'SA':
        time_slice = "2022-05-01/2022-08-31"
    if season == 'WS':
        time_slice = "2022-01-01/2022-04-30"

    vv_list = []
    vh_list = []
    vv_by_vh_list = []

    bbox_of_interest = [longitude , latitude, longitude, latitude]
    time_of_interest = time_slice

    catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    search = catalog.search(collections=["sentinel-1-rtc"], bbox=bbox_of_interest, datetime=time_of_interest)
    items = list(search.get_all_items())
    item = items[0]
    items.reverse()

    data = stac_load([items[1]],bands=bands_of_interest, patch_url=pc.sign, bbox=bbox_of_interest).isel(time=0)

    for item in items:
        data = stac_load([item], bands=bands_of_interest, patch_url=pc.sign, bbox=bbox_of_interest).isel(time=0)
        if(data['vh'].values[0][0]!=-32768.0 and data['vv'].values[0][0]!=-32768.0):
            data = data.where(~data.isnull(), 0)
            vh = data["vh"].astype("float64")
            vv = data["vv"].astype("float64")
            vv_list.append(np.median(vv))
            vh_list.append(np.median(vh))
            vv_by_vh_list.append(np.median(vv)/np.median(vh))

    return vv_list, vh_list, vv_by_vh_list

In [ ]:
## Get Sentinel-1-RTC Data
def get_sentinel_data_for_record(record):
    # Extract the longitude, latitude, and season from the record
    longitude = record['Longitude']
    latitude = record['Latitude']
    season = record['Season(SA = Summer Autumn, WS = Winter Spring)']

    assests = ['vh', 'vv']
    vh, vv, vv_by_vh = get_sentinel_data(longitude, latitude, season, assests)

    return (vh, vv, vv_by_vh)

pool = multiprocessing.Pool()
results = []
for result in tqdm(pool.imap(get_sentinel_data_for_record, crop_yield_data.to_dict('records')), total=len(crop_yield_data)):
    results.append(result)
pool.close()

vh = [x[0] for x in results]
vv = [x[1] for x in results]
vv_by_vh = [x[2] for x in results]
vh_vv_data = pd.DataFrame(list(zip(vh, vv, vv_by_vh)), columns=['vh_list', 'vv_list', 'vv/vh_list'])
#vh_vv_data = pd.read_csv("Sentinel1_data.csv")

In [ ]:
#calculate rvi using Sentinel-1 Data
def calculate_rvi(mean_vh, mean_vv):
    dop = mean_vv / (mean_vv + mean_vh)
    m = 1 - dop
    rvi = np.sqrt(dop) * (4 * mean_vh) / (mean_vv + mean_vh)
    return rvi

In [ ]:
def ordinal_distribution(data, dx=3, dy=1, taux=1, tauy=1, return_missing=False, tie_precision=None):
    def setdiff(a, b):

        a = np.asarray(a).astype('int64')
        b = np.asarray(b).astype('int64')

        _, ncols = a.shape

        dtype={'names':['f{}'.format(i) for i in range(ncols)],
            'formats':ncols * [a.dtype]}

        C = np.setdiff1d(a.view(dtype), b.view(dtype))
        C = C.view(a.dtype).reshape(-1, ncols)

        return(C)

    try:
        ny, nx = np.shape(data)
        data   = np.array(data)
    except:
        nx     = np.shape(data)[0]
        ny     = 1
        data   = np.array([data])

    if tie_precision is not None:
        data = np.round(data, tie_precision)

    partitions = np.concatenate(
        [
            [np.concatenate(data[j:j+dy*tauy:tauy,i:i+dx*taux:taux]) for i in range(nx-(dx-1)*taux)]
            for j in range(ny-(dy-1)*tauy)
        ]
    )

    symbols = np.apply_along_axis(np.argsort, 1, partitions)
    symbols, symbols_count = np.unique(symbols, return_counts=True, axis=0)

    probabilities = symbols_count/len(partitions)

    if return_missing==False:
        return symbols, probabilities

    else:
        all_symbols   = list(map(list,list(itertools.permutations(np.arange(dx*dy)))))
        miss_symbols  = setdiff(all_symbols, symbols)
        symbols       = np.concatenate((symbols, miss_symbols))
        probabilities = np.concatenate((probabilities, np.zeros(miss_symbols.__len__())))

        return symbols, probabilities

In [ ]:
def permutation_entropy(data, dx=3, dy=1, taux=1, tauy=1, base=2, normalized=True, probs=False, tie_precision=None):
    if not probs:
        _, probabilities = ordinal_distribution(data, dx, dy, taux, tauy, return_missing=False, tie_precision=tie_precision)
    else:
        probabilities = np.asarray(data)
        probabilities = probabilities[probabilities>0]

    if normalized==True and base in [2, '2']:
        smax = np.log2(float(np.math.factorial(dx*dy)))
        s    = -np.sum(probabilities*np.log2(probabilities))
        return s/smax

    elif normalized==True and base=='e':
        smax = np.log(float(np.math.factorial(dx*dy)))
        s    = -np.sum(probabilities*np.log(probabilities))
        return s/smax

    elif normalized==False and base in [2, '2']:
        return -np.sum(probabilities*np.log2(probabilities))
    else:
        return -np.sum(probabilities*np.log(probabilities))

In [ ]:
def generate_stastical_features(dataframe):
    features_list = []
    for index, row in dataframe.iterrows():

        min_vv = min(row[0])
        max_vv = max(row[0])
        range_vv = max_vv - min_vv
        mean_vv = np.mean(row[0])
        correlation_vv = sm.tsa.acf(row[0])[1]
        permutation_entropy_vv = permutation_entropy(row[0], dx=6,base=2, normalized=True)

        min_vh = min(row[1])
        max_vh = max(row[1])
        range_vh = max_vh - min_vh
        mean_vh = np.mean(row[1])
        correlation_vh = sm.tsa.acf(row[1])[1]
        permutation_entropy_vh = permutation_entropy(row[1], dx=6, base=2, normalized=True)

        min_vv_by_vh = min(row[2])
        max_vv_by_vh = max(row[2])
        range_vv_by_vh = max_vv_by_vh - min_vv_by_vh
        mean_vv_by_vh = np.mean(row[2])
        correlation_vv_by_vh = sm.tsa.acf(row[2])[1]
        permutation_entropy_vv_by_vh = permutation_entropy(row[2], dx=6, base=2, normalized=True)

        rvi = calculate_rvi(mean_vh, mean_vv)

        backscatter_coefficient = 10 * np.log10((mean_vv ** 2 + mean_vh ** 2) / 2)
        polarization = np.sqrt(mean_vv ** 2 + mean_vh ** 2)

        features_list.append([min_vv, max_vv, range_vv, mean_vv, correlation_vv, permutation_entropy_vv,
                          min_vh, max_vh, range_vh,  mean_vh, correlation_vh, permutation_entropy_vh,
                          min_vv_by_vh,  max_vv_by_vh, range_vv_by_vh, mean_vv_by_vh, correlation_vv_by_vh,
                          permutation_entropy_vv_by_vh, rvi, backscatter_coefficient, polarization])
    return features_list

In [ ]:
# Generating Statistical Features for VV,VH and VV/VH and creating a dataframe
features = generate_stastical_features(vh_vv_data)
features_data = pd.DataFrame(features, columns = ['min_vv', 'max_vv', 'range_vv', 'mean_vv', 'correlation_vv',
                                                  'permutation_entropy_vv','min_vh', 'max_vh', 'range_vh', 'mean_vh',
                                                  'correlation_vh', 'permutation_entropy_vh', 'min_vv_by_vh',
                                                  'max_vv_by_vh', 'range_vv_by_vh', 'mean_vv_by_vh',
                                                  'correlation_vv_by_vh', 'permutation_entropy_vv_by_vh', 'rvi',
                                                  'backscatter_coefficient', 'polarization'])

Sentinel-2

In [ ]:
def colorize(xx, colormap):
    return xr.DataArray(colormap[xx.data], coords=xx.coords, dims=(*xx.dims, "band"))

In [ ]:
def get_sentinel2_data(longitude, latitude, season, assets):
    '''
    Returns a list of RGB, NIR values for a given latitude and longitude over a given time period (based on the season)
    Attributes:
    bbox - bounding box
    time_range - time_of_interest
    '''
    bands_of_interest = assets
    if season == 'SA':
        time_slice = "2022-05-01/2022-08-31"
    elif season == 'WS':
        time_slice = "2022-01-01/2022-04-30"

    red_list = []
    green_list = []
    blue_list = []
    nir_list = []
    swir_list = []
    ndvi_list = []
    ndwi_list = []
    ndmi_list = []

    box_size_deg = 0.0004 # Surrounding box in degrees
    min_lon = longitude-box_size_deg/2
    min_lat = latitude-box_size_deg/2
    max_lon = longitude+box_size_deg/2
    max_lat = latitude+box_size_deg/2
    bounds = (min_lon, min_lat, max_lon, max_lat)

    catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    search = catalog.search(
        collections=["sentinel-2-l2a"],
        bbox=bounds,  # Pass the tuple instead of the Polygon object
        datetime=time_slice)
    items = list(search.get_all_items())

    resolution = 10  # meters per pixel
    scale = resolution / 111320.0 # degrees per pixel for CRS:4326

    xx = stac_load(
            items,
            bands=["red", "green", "blue", "nir", "swir16", "SCL"],
            crs="EPSG:4326", # Latitude-Longitude
            resolution=scale, # Degrees
            chunks={"x": 2048, "y": 2048},
            dtype="uint16",
            patch_url=pc.sign,
            bbox=bounds)

    # Create a colormap to display the SCL pixel classifications
    scl_colormap = np.array(
        [
            [252,  40, 228, 255],  # 0  - NODATA - MAGENTA
            [255,   0,   4, 255],  # 1  - Saturated or Defective - RED
            [0  ,   0,   0, 255],  # 2  - Dark Areas - BLACK
            [97 ,  97,  97, 255],  # 3  - Cloud Shadow - DARK GREY
            [3  , 139,  80, 255],  # 4  - Vegetation - GREEN
            [192, 132,  12, 255],  # 5  - Bare Ground - BROWN
            [21 , 103, 141, 255],  # 6  - Water - BLUE
            [117,   0,  27, 255],  # 7  - Unclassified - MAROON
            [208, 208, 208, 255],  # 8  - Cloud - LIGHT GREY
            [244, 244, 244, 255],  # 9  - Definitely Cloud - WHITE
            [195, 231, 240, 255],  # 10 - Thin Cloud - LIGHT BLUE
            [222, 157, 204, 255],  # 11 - Snow or Ice - PINK
        ],
        dtype="uint8",)

    # Load SCL band, then convert to RGB using color scheme above
    scl_rgba = colorize(xx.SCL.compute(), scl_colormap)
    # Create a mask for no data, saturated data, clouds, cloud shadows, and water
    cloud_mask = \
        (xx.SCL != 0) & \
        (xx.SCL != 1) & \
        (xx.SCL != 3) & \
        (xx.SCL != 6) & \
        (xx.SCL != 8) & \
        (xx.SCL != 9) & \
        (xx.SCL != 10)
    # Apply cloud mask ... NO Clouds, NO Cloud Shadows and NO Water pixels
    # All masked pixels are converted to "No Data" and stored as 16-bit integers
    cleaned_data = xx.where(cloud_mask).astype("uint16")
    # Load SCL band, then convert to RGB using color scheme above
    scl_rgba_clean = colorize(cleaned_data.SCL.compute(), scl_colormap)

    # Calculate the mean of the data across the sample region and then NDVI
    # Perform this calculation for the unfiltered and cloud-filtered (clean) datasets
    mean_unfiltered = xx.mean(dim=['longitude','latitude']).compute()
    ndvi_mean = (mean_unfiltered.nir-mean_unfiltered.red)/(mean_unfiltered.nir+mean_unfiltered.red)
    mean_clean = cleaned_data.mean(dim=['longitude','latitude']).compute()
    ndvi_mean_clean = (mean_clean.nir-mean_clean.red)/(mean_clean.nir+mean_clean.red)
    ndwi = (mean_clean.green - mean_clean.nir) / (mean_clean.green + mean_clean.nir)
    ndmi = (mean_clean.nir - mean_clean.swir16) / (mean_clean.nir + mean_clean.swir16)

    r = xx["red"].values
    g = xx["green"].values
    b = xx["blue"].values
    nir = xx["nir"].values
    swir = xx["swir16"].values

    # Apply cloud mask
    cloud_mask = np.isin(xx.SCL, [3, 8, 9])
    r = np.where(cloud_mask, np.nan, r)
    g = np.where(cloud_mask, np.nan, g)
    b = np.where(cloud_mask, np.nan, b)
    nir = np.where(cloud_mask, np.nan, nir)
    swir = np.where(cloud_mask, np.nan, swir)

    # Apply exposure adjustment
    r = exposure.adjust_log(r, gain=2)
    g = exposure.adjust_log(g, gain=2)
    b = exposure.adjust_log(b, gain=2)
    nir = exposure.adjust_log(nir, gain=2)
    swir = exposure.adjust_log(swir, gain=2)

    r_mean = np.nanmean(r)
    g_mean = np.nanmean(g)
    b_mean = np.nanmean(b)
    nir_mean = np.nanmean(nir)
    swir_mean = np.nanmean(swir)

    red_list.append(r_mean)
    green_list.append(g_mean)
    blue_list.append(b_mean)
    nir_list.append(nir_mean)
    swir_list.append(swir_mean)
    ndvi_list.append(ndvi_mean_clean)
    ndwi_list.append(ndwi)
    ndmi_list.append(ndmi)

    return red_list, green_list, blue_list, nir_list, swir_list, ndvi_list, ndwi_list, ndmi_list

In [ ]:
# Define the function to be used in multiprocessing
def get_sentinel2_data_for_bbox(record):
    latitude = record['Latitude']
    longitude = record['Longitude']
    season = record['Season(SA = Summer Autumn, WS = Winter Spring)']
    assets = ["red", "green", "blue", "nir", "swir16", "SCL"]
    red_list, green_list, blue_list, nir_list, swir_list, ndvi_list, ndwi_list, ndmi_list = get_sentinel2_data(longitude, latitude, season, assets)
    return red_list, green_list, blue_list, nir_list, swir_list, ndvi_list, ndwi_list, ndmi_list

# Create a multiprocessing pool and apply the function to each record in the crop_yield_data DataFrame
pool = multiprocessing.Pool()
results = []
for result in tqdm(pool.imap(get_sentinel2_data_for_bbox, crop_yield_data.to_dict('records')), total=len(crop_yield_data)):
    results.append(result)
pool.close()

In [ ]:
# Extract the RGB and NIR lists from the results

red_list = [x[0] for x in results]
green_list = [x[1] for x in results]
blue_list = [x[2] for x in results]
nir_list = [x[3] for x in results]
swir_list = [x[4] for x in results]
ndvi_list = [x[5] for x in results]
ndwi_list = [x[6] for x in results]
ndmi_list = [x[7] for x in results]

In [ ]:
#Extract features from Sentinel-2 Data
def extract_features(red_list, green_list, blue_list, nir_list, swir_list, ndvi_list, ndwi_list, ndmi_list):
    features2 = []
    for i in range(max(len(red_list),len(green_list),len(blue_list),len(nir_list),len(ndvi_list),len(ndwi_list))):
        red = red_list[i]
        green = green_list[i]
        blue = blue_list[i]
        nir = nir_list[i]
        swir = swir_list[i]
        ndvi = ndvi_list[i]
        ndwi = ndwi_list[i]
        ndmi = ndmi_list[i]

        r_mean = np.mean(red)
        g_mean = np.mean(green)
        b_mean = np.mean(blue)
        nir_mean = np.mean(nir)
        swir_mean = np.mean(swir)

        red_mean = (2 * r_mean + g_mean) / 3
        blue_mean = (2 * b_mean + g_mean) / 3
        green_mean = (r_mean + g_mean + b_mean) / 3
        brightness = np.sqrt(0.299 * r_mean**2 + 0.587 * g_mean**2 + 0.114 * b_mean**2)

        img = np.array([r_mean, g_mean, b_mean, nir_mean]).reshape((2, 2))
        img = (img * 255).astype(np.uint8)  # Convert to uint8
        glcm = graycomatrix(img, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)
        contrast = graycoprops(glcm, 'contrast').mean()
        correlation = graycoprops(glcm, 'correlation').mean()
        energy = graycoprops(glcm, 'energy').mean()
        homogeneity = graycoprops(glcm, 'homogeneity').mean()

        features2.append([r_mean, g_mean, b_mean, nir_mean, swir_mean, ndvi, ndwi, ndmi,
                          red_mean, blue_mean, green_mean, brightness, contrast, correlation, energy, homogeneity])

    return features2

In [ ]:
# Generating Statistical Features for RGB and NIR and creating a dataframe
features2 = extract_features(red_list, green_list, blue_list, nir_list, swir_list, ndvi_list, ndwi_list, ndmi_list)
features2_data = pd.DataFrame(features2, columns = ['r_mean', 'g_mean', 'b_mean', 'nir_mean', 'swir_mean', 'ndvi', 'ndwi',
                                                    'ndmi', 'red_mean', 'blue_mean', 'green_mean', 'brightness',
                                                    'contrast', 'correlation', 'energy', 'homogeneity'])
features2_data.head()

In [ ]:
#Convert NDVI and NDWI xarray to nparray
ndvi_array = np.array([da for da in features2_data['ndvi']])
ndvi_values = [da[0][0].item() for da in ndvi_array]
ndvi = ndvi_values

ndwi_array = np.array([da for da in features2_data['ndwi']])
ndwi_values = [da[0][0].item() for da in ndwi_array]
ndwi = ndwi_values

ndmi_array = np.array([da for da in features2_data['ndmi']])
ndmi_values = [da[0][0].item() for da in ndmi_array]
ndmi = ndmi_values

In [ ]:
# Create a new DataFrame with updated columns
features2_data['ndvi'] = ndvi
features2_data['ndwi'] = ndwi
features2_data['ndmi'] = ndmi
features2_data.head()

Combine two data

In [ ]:
def combine_two_datasets(dataset1,dataset2):
    data = pd.concat([dataset1,dataset2], axis=1)
    return data

In [ ]:
features_data = combine_two_datasets(features_data, features2_data)
features_data.head()

In [ ]:
features_data.to_csv('Features1_data.csv', index=False)

In [ ]:
crop_data = combine_two_datasets(crop_yield_data,features_data)
crop_data.head()

In [ ]:
#Take all columns of Features_data and Features2_data
crop_data = crop_data[['min_vv', 'max_vv', 'range_vv', 'mean_vv', 'correlation_vv', 'permutation_entropy_vv',
                       'min_vh', 'max_vh', 'range_vh', 'mean_vh', 'correlation_vh', 'permutation_entropy_vh',
                       'min_vv_by_vh',  'max_vv_by_vh', 'range_vv_by_vh', 'mean_vv_by_vh', 'correlation_vv_by_vh',
                       'permutation_entropy_vv_by_vh', 'rvi', 'backscatter_coefficient', 'polarization',

                       'r_mean', 'g_mean', 'b_mean', 'nir_mean', 'swir_mean', 'ndvi', 'ndwi', 'ndmi',
                       'red_mean', 'blue_mean', 'green_mean', 'brightness', 'contrast', 'correlation',
                       'energy', 'homogeneity' 'Field size (ha)', 'Rice Yield (kg/ha)']]
crop_data.head()

In [ ]:
#correlation matrix
corrmat = crop_data.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True)

In [ ]:
#Use columns correlated with Rice Yield
crop_data = crop_data[['permutation_entropy_vv','permutation_entropy_vh','correlation_vv', 'correlation_vh',
                       'permutation_entropy_vv_by_vh', 'correlation_vv_by_vh', 'rvi', 'backscatter_coefficient', 'polarization',

                       'ndvi', 'ndwi', 'contrast', 'energy',

                       'Rice Yield (kg/ha)']]
crop_data.head()

In [ ]:
# Drop rows with all missing values in training and validation data
crop_data = crop_data.dropna(axis=0, how='any')

#Check if there is missing value
missing_val_count_by_column = (crop_data.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

In [ ]:
# Split data into features and target
X = crop_data.drop('Rice Yield (kg/ha)', axis=1)
y = crop_data['Rice Yield (kg/ha)']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(X_train_scaled.size)
print(X_test_scaled.size)